In [23]:
from tbparse import SummaryReader
import torch
# import required module
import os 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# load one from prompt learning
log_dir = "../prompt-based-models/logs/icd9_triage/emilyalsentzer/Bio_ClinicalBERT_tempmanual2_verbmanual0/version_23-02-2022--10-26/events.out.tfevents.1645611997.doraemon1.19356.0"
reader = SummaryReader(log_dir)
df = reader.scalars
print(df)

       step                  tag     value
0         0     train/batch_loss  3.452232
1         1     train/batch_loss  3.610268
2         2     train/batch_loss  3.639858
3         3     train/batch_loss  3.375153
4         4     train/batch_loss  3.340623
...     ...                  ...       ...
11999     9         valid/recall  0.946050
12000     0   zero_shot/accuracy  0.135242
12001     0         zero_shot/f1  0.050305
12002     0  zero_shot/precision  0.031419
12003     0     zero_shot/recall  0.128625

[12004 rows x 3 columns]


2022-02-23 13:11:15.473032: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda-9.0/lib64
2022-02-23 13:11:15.476379: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-02-23 13:11:15.477365: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild Ten

In [3]:
df[df["tag"]=="valid/f1"]

,step,tag,value
11970,0,valid/f1,0.943957
11971,1,valid/f1,0.945584
11972,2,valid/f1,0.934010
11973,3,valid/f1,0.944772
11974,4,valid/f1,0.945066
11975,5,valid/f1,0.944072
11976,6,valid/f1,0.943878
11977,7,valid/f1,0.943819
11978,8,valid/f1,0.946914
11979,9,valid/f1,0.945518


In [13]:
'frozen_plm' in log_dir

True

In [15]:
'tempmixed' in log_dir

True

In [4]:
# load one from clinical longformer pipeline

log_dir = "/mnt/sdg/niallt/saved_models/mimic-tasks/bert-based-models/logs/icd9_triage/frozen_plm/emilyalsentzer/Bio_ClinicalBERT/version_22-02-2022--17-31-12/"
reader = SummaryReader(log_dir)
df = reader.scalars
print(df)

      step           tag     value
0       49         epoch  0.000000
1       99         epoch  0.000000
2      149         epoch  0.000000
3      199         epoch  0.000000
4      249         epoch  0.000000
...    ...           ...       ...
3055     5  valid/recall  0.918112
3056     6  valid/recall  0.918433
3057     7  valid/recall  0.920360
3058     8  valid/recall  0.922286
3059     9  valid/recall  0.923571

[3060 rows x 3 columns]


In [5]:
df[df["tag"]=="valid/f1"]

,step,tag,value
3027,0,valid/f1,0.100000
3028,0,valid/f1,0.818529
3029,1,valid/f1,0.881736
3030,2,valid/f1,0.873895
3031,3,valid/f1,0.908225
3032,4,valid/f1,0.912533
3033,5,valid/f1,0.913972
3034,6,valid/f1,0.912850
3035,7,valid/f1,0.915421
3036,8,valid/f1,0.915894


# TODO 

make funciton that will interatively read in tensorboard events files and split the filename to create certain variables such as 
template type and whether plm was frozen


In [5]:
log_dir = "../prompt-based-models/logs/icd9_triage/"

In [18]:

dfs = []
# that directory
for root, dirs, files in os.walk(log_dir):
    for filename in files:
        full_path = os.path.join(root, filename)       
        
        # the file name for tensorboard logs is horrible so just regex for tfevents
        if "tfevents" in full_path:        
            
            # now parse the tensorboard summary data
            reader = SummaryReader(file)
            df = reader.scalars

            # get the model paramters based on filename
            # Was plm frozen?
            if 'frozen_plm' in full_path:
                df["plm_frozen"] = True
            else:
                df["plm_frozen"] = True

            # which template type
            if 'tempmanual' in full_path:
                df["temp_type"] = "manual"
            elif 'tempsoft' in full_path:
                df["temp_type"] = "soft"
                
            dfs.append(df)
            


In [24]:
pd.concat(dfs)

,step,tag,value,plm_frozen,temp_type
0,0,train/batch_loss,3.742105,True,manual
1,1,train/batch_loss,3.694839,True,manual
2,2,train/batch_loss,3.936922,True,manual
3,3,train/batch_loss,3.802533,True,manual
4,4,train/batch_loss,3.494807,True,manual
...,...,...,...,...,...
2291,0,valid/recall,0.893064,True,soft
2292,0,zero_shot/accuracy,0.132030,True,soft
2293,0,zero_shot/f1,0.062982,True,soft
2294,0,zero_shot/precision,0.444971,True,soft


In [70]:
class TB_DataProcessor:
    def __init__(self,log_dir):
        # set the log dir 
        self.log_dir = log_dir
        
        # concatenate all the log files in the directory 
        self.all_logs = self.concatenate_all_logs(self.log_dir)       
            
    
    def concatenate_all_logs(self, log_dir):        
       
        print("concatenating all log files!")
        # empy list to fill
        dfs = []
        # that directory
        for root, dirs, files in os.walk(log_dir):
            for filename in files:
                full_path = os.path.join(root, filename)       

                # the file name for tensorboard logs is horrible so just regex for tfevents
                if "tfevents" in full_path:        

                    # now parse the tensorboard summary data
                    reader = SummaryReader(file)
                    df = reader.scalars

                    # get the model paramters based on filename
                    # Was plm frozen?
                    if 'frozen_plm' in full_path:
                        df["plm_frozen"] = True
                    else:
                        df["plm_frozen"] = True

                    # which template type
                    if 'tempmanual' in full_path:
                        df["temp_type"] = "manual"
                    elif 'tempsoft' in full_path:
                        df["temp_type"] = "soft"
                    
                    # append individual df to list
                    dfs.append(df)    
        
        # concat all the dfs
        all_logs = pd.concat(dfs)
            
        return all_logs
    
    def extract_metric(self, mode = "all",metrics = "all"):
        '''
        Function to pull certain specified features from the full logs dataframes
        
        Args: 
            mode: The dataset you want metrics for e.g. train/valid/test
            metrics: The metric you want to look at e.g. f1, precision etc.
        '''
        # get the logs to work with
        all_logs = self.all_logs
        if metrics == "all" and mode == "all":
            return logs
        
        # now for cases where mode is specific        
        elif metrics == "all" and mode != "all":            
            return all_logs[all_logs["tag"].str.contains(mode)]
        
        # now for cases where both metric is specific
        
        elif metrics != "all" and mode == "all":
            return all_logs
            
            
        
        
        else:
            raise NotImplementedError
        
        

In [71]:
tb_data_processor = TB_DataProcessor(log_dir = "../prompt-based-models/logs/icd9_triage/")

concatenating all log files!


In [72]:
tb_data_processor.extract_metric(mode = "valid", metrics = "all")

,step,tag,value,plm_frozen,temp_type
7428,0,valid/balanced_accuracy,0.630231,True,manual
7429,1,valid/balanced_accuracy,0.830926,True,manual
7430,2,valid/balanced_accuracy,0.808761,True,manual
7431,3,valid/balanced_accuracy,0.832931,True,manual
7432,4,valid/balanced_accuracy,0.862978,True,manual
...,...,...,...,...,...
7447,1,valid/recall,0.921644,True,soft
7448,2,valid/recall,0.931278,True,soft
7449,3,valid/recall,0.933526,True,soft
7450,4,valid/recall,0.934489,True,soft
